prompt:
我想用别人训练好的模型去分辨我这些图片里是什么动物，从而辅助我这个模型更理解我给不同动物图片打分的逻辑
有没有一个能识别动物的model我能用的

预训练模型：我们使用了 VGG16 作为预训练的特征提取器。你可以根据需要选择其他预训练模型（如 ResNet、Inception 等），这些模型已经学会了如何从图像中提取高级特征。
特征提取：通过 base_model.predict() 获取每张图片的特征。flatten() 操作将多维特征图转为一维向量，方便后续处理。
评分模型：我们使用了一个简单的 线性回归 模型来预测每张图片的评分。你也可以使用更复杂的机器学习或深度学习模型。

1. 加载预训练的 VGG16 模型

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import pandas as pd
import os

In [2]:
# 加载预训练的VGG16模型（不包含顶层分类层）
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [3]:
# 冻结模型的卷积层
base_model.trainable = False

2. 定义特征提取函数

In [4]:
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    features = base_model.predict(img_array)  # 获取特征
    features = features.flatten()  # 将特征展平成一维向量
    return features

3. 载入你的图片并提取特征

In [5]:
data_dir = "./animal_pics"  # 图片文件夹路径
image_files = os.listdir(data_dir)
features_list = []

# 提取每张图片的特征
for img_file in image_files:
    img_path = os.path.join(data_dir, img_file)
    features = extract_features(img_path)
    features_list.append(features)

# 将特征转化为 DataFrame
features_df = pd.DataFrame(features_list)

# 假设 CSV 文件里有图片的分数
df = pd.read_csv("elo_scores.csv")  # 包含图片文件名和评分的 CSV 文件
df['features'] = features_df.values.tolist()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 640ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 623ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 648ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 653ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 645ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 658ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 671ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 673ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 662ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 690ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 695ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 668ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 639ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 656ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 661ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 646ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 674ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 674ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 648ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 657ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 660ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 657ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

4. 构建你自己的模型
现在你可以将提取的特征与图片的分数结合，来训练你的模型。例如，可以使用回归模型来预测图片的评分。

In [6]:
from sklearn.linear_model import LinearRegression

In [8]:
# 准备数据
X = np.array(df['features'].tolist())  # 特征
y = df['Score'].values  # 分数

In [9]:
# 创建回归模型
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [10]:
# 保存训练好的模型
import pickle
with open('scoring_model.pkl', 'wb') as f:
    pickle.dump(model, f)

5. 使用训练好的评分模型预测新图片的评分

In [14]:
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    features = base_model.predict(img_array)  # 获取特征
    features = features.flatten()  # 将特征展平成一维向量
    return features

In [15]:
def predict_score(img_path, scoring_model):
    features = extract_features(img_path)
    score = scoring_model.predict([features])
    return score[0]

In [18]:
def compare_images(img_path1, img_path2, scoring_model):
    score1 = predict_score(img_path1, scoring_model)
    score2 = predict_score(img_path2, scoring_model)
    
    if score1 > score2:
        print(f"Image 1 ({img_path1}) has a higher score: {score1}")
        print(f"Image 2 ({img_path2}) has a lower score: {score2}")
        return img_path1
    else:
        print(f"Image 2 ({img_path2}) has a higher score: {score2}")
        print(f"Image 1 ({img_path1}) has a lower score: {score1}")
        return img_path2

In [21]:
img1_path = './animal_pics/google-advancedsearch-27.jpg'
img2_path = './animal_pics/google-advancedsearch-52.jpg'

# 加载评分模型
with open('scoring_model.pkl', 'rb') as f:
    scoring_model = pickle.load(f)

# 比较两张图片
best_image = compare_images(img1_path, img2_path, scoring_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
Image 2 (./animal_pics/google-advancedsearch-52.jpg) has a higher score: 1021.0151560000002
Image 1 (./animal_pics/google-advancedsearch-27.jpg) has a lower score: 948.3898928000002
